# 1. Getting VizDoom Up and Running

In [ ]:
!pip install vizdoom

In [ ]:
!cd github & git clone https://github.com/Farama-Foundation/ViZDoom

In [9]:
#Import vizdoom for game env
from vizdoom import *
#Import random for action sampling
import random
#Import time for sleeping b/w frames
import time
#Import numpy for identity matrix
import numpy as np

In [2]:
#Setup game
game = DoomGame()
game.load_config('github/VizDoom/scenarios/deadly_corridor.cfg')
game.init()

In [3]:
#This is the set of actions we can take in the environment
actions = np.identity(7, dtype=np.uint8)

In [ ]:
state = game.get_state()

In [ ]:
state.game_variables

In [ ]:
#Loop thorugh episodes
episodes = 10
for episode in range(episodes):
    #Create a new episode or game
    game.new_episode()
    #Checking hte game isn't finish
    while not game.is_episode_finished():
        #Get the game state
        state = game.get_state()
        #Get the game image
        img = state.screen_buffer
        #Get the game variables - ammo
        info = state.game_variables
        #Take an action
        reward = game.make_action(random.choice(actions),4)
        #Print reward
        print('Reward:', reward)
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

In [ ]:
game.close()


# 2. Converting it to a Gym Environment

In [ ]:
!pip install gym

In [10]:
#Import environment base class from OpenAI Gym
import gymnasium as gym

#Import gym spaces
from gymnasium.spaces import Discrete, Box
#Import opencv
import cv2
#Import vizdoom for game env
from vizdoom import *
#Import random for action sampling
import random
#Import time for sleeping b/w frames
import time
#Import numpy for identity matrix
import numpy as np

In [11]:
#Create Vizdoom OpenAI Gym Environment
class VizDoomGym(gym.Env):
    #Function that is called when we start the env
    def __init__(self, render=False, config='github/VizDoom/scenarios/deadly_corridor.cfg'):
        #Inherit from Env
        super().__init__()
        #Setup the game
        self.game = DoomGame()
        self.game.load_config(config)

        #Render frame Logic
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
       
        #Start the game
        self.game.init() 
       
        #Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)

        #Game variables: HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 60
        
    #This is how we take a step in the env    
    def step(self, action):
        #Specify action and take step
        actions = np.identity(3, dtype=np.uint8)
        movement_reward = self.game.make_action(actions[action], 4)
       
        reward = 0
        #Get all the other stuff we need to reutrn
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            
            #Reward shaping
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables

            #Calculate reward deltas
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo

            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200 + ammo_delta*5          
            info = ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        truncated = False

        return state, reward, done, truncated, info
        
    #Define how to render the game or environment
    def render():
        pass
    #What happen when we start a new game
    def reset(self, seed=None):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        ammo = self.game.get_state().game_variables[0]
        info = {"ammo":ammo}
        return self.grayscale(state), info
        
    #Grayscale the game frame and resize it
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
        
    #Call to close down the game
    def close(self):
        self.game.close()

In [10]:
env = VizDoomGym(render=True)

In [ ]:
state = env.reset()

In [ ]:
env.close()

In [7]:
#Import Environment checker
from stable_baselines3.common import env_checker

In [11]:
env_checker.check_env(env)

# 3. View State

In [ ]:
!pip install matplotlib


In [8]:
from matplotlib import pyplot as plt

In [ ]:
grayscale_image, info = env.reset()
plt.imshow(grayscale_image.squeeze(), cmap='gray')


In [ ]:
env.close()

# 4. Setup Callback

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install stable-baselines3[extra]

In [12]:
#Import os for file nav
import os
#Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [13]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [14]:
CHECKPOINT_DIR = './train/train_corridor'
LOG_DIR = './logs/log_corridor'

In [15]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# 5. Train Model Using Curriculum

In [22]:
#import PPO for training
from stable_baselines3 import PPO

In [23]:
#Non rendered environment
env = VizDoomGym(config='github/VizDoom/scenarios/deadly_corridor_s1.cfg')

In [ ]:
env.close()


In [24]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=.95, gae_lambda=.9)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
model.load('./train/train_corridor/best_model_80000.zip')

In [ ]:
env = VizDoomGym(config='github/VizDoom/scenarios/deadly_corridor_s2.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/VizDoom/scenarios/deadly_corridor_s3.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/VizDoom/scenarios/deadly_corridor_s4.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
env = VizDoomGym(config='github/VizDoom/scenarios/deadly_corridor_s5.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

# 6. Test the Model

In [ ]:
#Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
#Reload model from disc
model = PPO.load('./train/train_defend/best_model_100000')

In [ ]:
#Create rendered environment
env = VizDoomGym(render=True)

In [ ]:
#Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=300)

In [ ]:
mean_reward

In [ ]:
print(env.reset())


In [ ]:
obs = env.reset()
print(type(obs))
if isinstance(obs, tuple):
    for i, item in enumerate(obs):
        print(f"Item {i} type: {type(item)}, shape: {getattr(item, 'shape', 'N/A')}")
else:
    print(f"Observation shape: {obs.shape}")


In [ ]:
for episode in range(5):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        time.sleep(0.05)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)